# project_don_ross

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://donrossonline.com/tunings/'

In [3]:
r = requests.get(url)

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
results = soup.find_all('span', attrs={'class': 'music-tune'})

In [6]:
tunings = []

for row in results:
    tunings.append(row.contents[0])
    
tunings = pd.Series(tunings)
tunings.head()

0      DADGBD
1      DADGCE
2      EBDEBE
3      AEADEA
4    G#EADF#B
dtype: object

In [7]:
tunings = tunings.str.extract(r'([ABCDEFG#b]{6,14})', expand=False)
tunings.dropna(inplace=True)
tunings.head()

0      DADGBD
1      DADGCE
2      EBDEBE
3      AEADEA
4    G#EADF#B
dtype: object

In [8]:
def separate_strings(s):
    '''
    takes a series (s) of guitar tuning strings
    and returns a series of lists (x) showing 
    individual strings including accidentals (#, b)
    '''
    strings = list(s)
    for i in range(len(strings)):
        if strings[i] in list('#b'):
            '''
            doesn't throw IndexError but can produce
            odd results if tuning entered incorrectly
            (e.g. accidental before string name)
            '''
            strings[i-1] = strings[i-1] + strings[i]

    x = []
    for c in strings:
        if c not in list('#b'):
            x.append(c)
        
    return x

In [9]:
tunings = tunings.apply(separate_strings)
tunings.head()

0      [D, A, D, G, B, D]
1      [D, A, D, G, C, E]
2      [E, B, D, E, B, E]
3      [A, E, A, D, E, A]
4    [G#, E, A, D, F#, B]
dtype: object

In [10]:
def show_tunings(s, num_strings):
    '''
    takes a series of lists (s) of guitar tunings
    and returns a series of string guitar tunings
    based on the number of strings (num_strings) 
    inputted. nlargest(10) is returned
    '''
    return s.apply(lambda x: ''.join(x[0:num_strings])).value_counts().nlargest(10)

In [11]:
# 10 most popular Don Ross tunings - bottom 3 strings 
show_tunings(tunings, 3)

DAD     27
EAD     22
BEA      9
CGC      9
DAC      7
AEA      6
FAC      5
CGD      5
CGEb     4
EBD      3
dtype: int64

In [12]:
# 10 most popular Don Ross tunings - bottom 4 strings
show_tunings(tunings, 4)

DADG     23
EADG     21
BEAD      9
CGCG      6
CGDG      5
AEAD      4
DACF      4
CGEbF     4
FACF      3
CGCF      3
dtype: int64

In [13]:
# 10 most popular Don Ross tunings - bottom 5 strings
show_tunings(tunings, 5)

EADGB     19
DADGB     14
BEADF#     7
DADGA      6
CGDGB      4
DACFC      4
FACFC      3
DACGC      3
DADGC      3
CGCGC      3
dtype: int64

In [14]:
# 10 most popular Don Ross tunings - bottom 6 strings
show_tunings(tunings, 6)

EADGBE        19
DADGBE        12
BEADF#B        7
DADGAD         6
CGEbFBbD       3
BF#C#F#BF#     2
CGDGBE         2
BEADGB         2
DACFCF         2
DAEF#AE        2
dtype: int64